### Importing Libraries

In [1]:
# data manipulation and visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# scraping libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException, ElementNotVisibleException,\
NoSuchElementException, StaleElementReferenceException, NoSuchAttributeException

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Amazon Reviews

In [6]:
driver = webdriver.Chrome(r"C:/Program Files (x86)/chromedriver.exe")
driver.maximize_window()

# go to amazon
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)

# search for the Smart Watches, Cameras, Printers, Monitors, TVs and Routers
srch_items = ['smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']
title = []
review_text = []
ratings = []

for i in srch_items:
    srchbar = driver.find_element(By.ID, "twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element(By.ID, "nav-search-submit-button").click()  #clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements(By.XPATH, "//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href')) # gets the url of the item
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element(By.XPATH, "//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        # getting the reviews based on the stars they've received
        url_ = []
        try:
            two_str = driver.find_element(By.XPATH, "//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element(By.XPATH, "//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element(By.XPATH, "//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element(By.XPATH, "//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element(By.XPATH, "//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements(By.XPATH, "//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                try:
                    for i in ids:
                        comment_ids.append(i.get_attribute('id'))
                except StaleElementReferenceException:
                    pass

                for x in comment_ids:
                    # Extract user IDs from each user on a page
                    try:
                        review_title = driver.find_element(By.XPATH, '//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')
                    except StaleElementReferenceException:
                        pass

                    try:
                        rating = driver.find_element(By.XPATH, '//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')
                    except StaleElementReferenceException:
                        pass

                    try:
                        text = driver.find_element(By.XPATH, '//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                    except StaleElementReferenceException:
                        pass
                    
                try:
                    driver.find_element(By.XPATH, "//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 
len(ratings), len(title), len(review_text)

(44117, 44118, 44122)

In [7]:
# Creating a dataframe
data = list(zip(title,review_text,ratings))
Amazon_reviews = pd.DataFrame(data, columns = ["Review_title","Review_text","Ratings"])
Amazon_reviews

,Review_title,Review_text,Ratings
0,Nothing more than a traditional watch,The product is just like traditional watch. It...,2.0 out of 5 stars
1,DONT BUY GO WITH OTHER BRAND,ACCURACY OF THIS WATCH IS TOTALLY WRONG.<br>BE...,2.0 out of 5 stars
2,Not as expected,"Sometimes touch doesn't work properly, looks l...",2.0 out of 5 stars
3,Disappointed with accuracy and interface.,Quite good product look and touch response is ...,2.0 out of 5 stars
4,Its good but not as expected,1. The steps are not counted everytime<br>2. T...,2.0 out of 5 stars
...,...,...,...
44112,,Works really well when connected. RV using WeB...,
44113,,Nice and handy. I use it in a campground that ...,
44114,,manual doesn't explain how to connect usb hard...,
44115,,The hard part first: Setup is difficult becau...,


In [46]:
# checking to see how many empty values are present in the Reveiw_title column
list_values = [x for x in Amazon_reviews['Review_title'].unique()]
list_values[10]

''

In [47]:
count = 0
for i in Amazon_reviews['Review_title']:
    if i == list_values[10]: count += 1
print (f"The amount of empty strings inn Review title is: {count}")

The amount of empty strings inn Review title is: 7763


In [65]:
df_amazon = Amazon_reviews.copy()
df_amazon = df_amazon[df_amazon["Review_title"].str.contains(' ') == True]
df_amazon

,Review_title,Review_text,Ratings
0,Nothing more than a traditional watch,The product is just like traditional watch. It...,2.0 out of 5 stars
1,DONT BUY GO WITH OTHER BRAND,ACCURACY OF THIS WATCH IS TOTALLY WRONG.<br>BE...,2.0 out of 5 stars
2,Not as expected,"Sometimes touch doesn't work properly, looks l...",2.0 out of 5 stars
3,Disappointed with accuracy and interface.,Quite good product look and touch response is ...,2.0 out of 5 stars
4,Its good but not as expected,1. The steps are not counted everytime<br>2. T...,2.0 out of 5 stars
...,...,...,...
44052,Great router,Good and easy,4.0 out of 5 stars
44058,Wrong iteams ordered so,Very bad,4.0 out of 5 stars
44059,PLEASE HELP !!!!! URGENT,Great item,4.0 out of 5 stars
44061,Good product,,4.0 out of 5 stars


In [66]:
# saving the amazon dataframe
df_amazon.to_csv('amazon_reviews',index=False)

In [12]:
# changing the rating column to only contain the actual number 
rating = [str(x)[0] for x in df_amazon.Ratings]
df_amazon.Ratings = rating

In [16]:
# checking to see if there's any unwanted variable in the column
df_amazon.Ratings.unique()

array(['2', '3', '1', '5', '4', 'n'], dtype=object)

In [18]:
# there seems to be a string "n" in the "Ratings" column, so we'll remove it
df_amazon = df_amazon[df_amazon["Ratings"].str.contains('n') == False]

,Review_title,Review_text,Ratings
0,Nothing more than a traditional watch,The product is just like traditional watch. It...,2
1,DONT BUY GO WITH OTHER BRAND,ACCURACY OF THIS WATCH IS TOTALLY WRONG.<br>BE...,2
2,Not as expected,"Sometimes touch doesn't work properly, looks l...",2
3,Disappointed with accuracy and interface.,Quite good product look and touch response is ...,2
4,Its good but not as expected,1. The steps are not counted everytime<br>2. T...,2
...,...,...,...
31926,Good product,There is some loss of data packets between the...,4
31927,Great router,Good and easy,4
31928,Wrong iteams ordered so,Very bad,4
31929,PLEASE HELP !!!!! URGENT,Great item,4


In [22]:
# looking for null values
df_amazon.isnull().sum()

Review_title       0
Review_text     2566
Ratings            0
dtype: int64

In [24]:
# dropping null values
df_amazon = df_amazon.dropna()

__Changes Made:__
- Dropped null values 
- Cleaned the `Ratings` column
- Removed rows containing unwanted data

In [28]:
# updating the saved file
df_amazon.to_csv('amazon_reviews',index=False)

### Flipkart Reviews

In [37]:
driver = webdriver.Chrome(r"C:/Program Files (x86)/chromedriver.exe")
driver.maximize_window()

# go to flipkart.com 
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)

#closing the pop up (if present)
try:
    driver.find_element(By.XPATH,"//div[@class='_2QfC02']/button").click()
except ElementNotInteractableException or ElementNotVisibleException: pass

srch_items = ['Laptops', 'Phones','Smart Watches','Monitors']
title = []
review_text = []
ratings = []
     
# creating a user defined function to scrape data (review text, review title and rating)
def scrape():    
#         scrape the text
        for i in driver.find_elements(By.XPATH,"//div[@class='t-ZTKy']"):
            review_text.append(i.text)
#         scrape the title of the review
        for i in driver.find_elements(By.XPATH,"//p[@class='_2-N8zT']"):
            title.append(i.text)
#         scrape the rating of the review
        for i in driver.find_elements(By.XPATH,"//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return
    
urls=[]
for i in srch_items:
    # Find the search bar
    srchBar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking the search button
    driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    
#     going through the pages present for the searched item
    page = []
    for i in driver.find_elements(By.XPATH,"//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))
print (f"The amount of URLs gathered is: {len(urls)}")            

# going to each url
for i in urls[:100]:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
    # Clicking on "all reviews"
    try:
        btn = driver.find_element(By.XPATH,"//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException: pass

    scrape()        
    try:
        n_page=driver.find_elements(By.XPATH,"//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrape()
    except: continue        

for i in urls[101:200]:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
    # Clicking on all reviews
    try:
        btn=driver.find_element(By.XPATH,"//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrape()        
    try:
        n_page=driver.find_elements(By.XPATH,"//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrape()
    except: continue
print (f"Review Tiles Scraped:{len(title)}\nReveiws Scraped:{len(review_text)}\
\nRatings Scraped:{len(ratings)}")        

The amount of URLs gathered is: 288
Review Tiles Scraped:17090
Reveiws Scraped:17090
Ratings Scraped:17090


In [50]:
driver = webdriver.Chrome(r"C:/Program Files (x86)/chromedriver.exe")
driver.maximize_window()

for i in urls[200:288]:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
    # Clicking on all reviews
    try:
        btn=driver.find_element(By.XPATH,"//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrape()
    try:
        n_page=driver.find_elements(By.XPATH,"//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue
print (f"Review Tiles Scraped:{len(title)}\nReveiws Scraped:{len(review_text)}\
\nRatings Scraped:{len(ratings)}")

Review Tiles Scraped:17838
Reveiws Scraped:17838
Ratings Scraped:17838


In [51]:
# Creating a dataframe
data_flipkart = list(zip(title,review_text,ratings))
flipkart_reviews = pd.DataFrame(data_flipkart, columns = ["Review_title","Review_text","Ratings"])
flipkart_reviews

,Review_title,Review_text,Ratings
0,Nice product,Writing this after using Vivobook 15 for 3 day...,4
1,Good choice,Day2\nPros\nDesign looks good\nPerformance til...,4
2,Classy product,Very good for usage after 1 month it is good a...,5
3,Moderate,If you can affort few thousands extra you can ...,2
4,Super!,Design looks good\nPerformance till now for ba...,5
...,...,...,...
17833,Fabulous!,Good monitor + ame + point,5
17834,Great product,This is awesome,5
17835,Simply awesome,Goof,5
17836,Worth every penny,Vary nice,5


In [52]:
df_flipkart = flipkart_reviews.copy()
# checking for null values 
df_flipkart.isnull().sum()

Review_title    0
Review_text     0
Ratings         0
dtype: int64

In [53]:
# saving the dataframe
df_flipkart.to_csv('flipkart_reviews',index=False)

### Combining the Dataframes

In [54]:
reviews = pd.concat([df_amazon, df_flipkart], ignore_index = True)
reviews

,Review_title,Review_text,Ratings
0,Nothing more than a traditional watch,The product is just like traditional watch. It...,2
1,DONT BUY GO WITH OTHER BRAND,ACCURACY OF THIS WATCH IS TOTALLY WRONG.<br>BE...,2
2,Not as expected,"Sometimes touch doesn't work properly, looks l...",2
3,Disappointed with accuracy and interface.,Quite good product look and touch response is ...,2
4,Its good but not as expected,1. The steps are not counted everytime<br>2. T...,2
...,...,...,...
47199,Fabulous!,Good monitor + ame + point,5
47200,Great product,This is awesome,5
47201,Simply awesome,Goof,5
47202,Worth every penny,Vary nice,5


In [55]:
# Saving the final data into csv file
reviews.to_csv("Reviews.csv")

In [56]:
df = reviews.copy()

In [57]:
df.Reveiw_title.unique()

array(['2', '3', '1', '5', '4', 'n'], dtype=object)

In [59]:
# checking the amount of times
(reviews=='n').sum()

Review_title      0
Review_text       0
Ratings         901
dtype: int64

In [ ]:
# there seems to be a string "n" in the "Ratings" column, so we'll remove it
df_amazon = df_amazon[df_amazon["Ratings"].str.contains('n') == False]